In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
import os
print(os.getcwd())

/content


# Son Tran Code


In [ ]:
# Sơn's code
# Sơn's code
# Sơn's code
import os
import numpy as np
import random
import torch
def load_glove_vectors(glove_file):
    with open(glove_file, 'r', encoding="utf-8") as file:
        # unique words
        words = {}
        word_to_vec = []
        # each line starts with a word then the values for the different features
        for index, line in enumerate(file):
            line = line.strip().split()
            # take the word 
            curr_word = line[0]
            words[curr_word] = index
            # rest of the features for the word
            word_to_vec.append(np.array(line[1:], dtype=np.float64))
        word_to_vec = np.stack(word_to_vec, axis=0)

        word_to_vec = torch.from_numpy(word_to_vec)
            
    return words, word_to_vec
words, word_to_vec = load_glove_vectors('gdrive/MyDrive/NLP-2021/Project1-NLP/WordAnalogy-GLoVe/data/glove.6B.300d.txt')


def find_cosine_similarity(word_to_vec, v):
    
    # find the dot product between u and v 
    dot = torch.matmul(word_to_vec, torch.transpose(v.unsqueeze(0),0,1))
    # find the L2 norm of u 
    norm_u = torch.sqrt(torch.sum(word_to_vec**2, dim=1))
    # Compute the L2 norm of v
    norm_v = torch.sqrt(torch.sum((v**2).unsqueeze(0), dim=1))
    # Compute the cosine similarity
    norm_u = norm_u.unsqueeze(1)
    cosine_sim = dot/norm_u/norm_v
    
    return cosine_sim

def find_analogy(word_a, word_b, word_c, words, word_to_vec):
    # convert words to lower case
    word_a = word_a.lower()
    word_b = word_b.lower()
    word_c = word_c.lower()
    
    
    # find the word embeddings for word_a, word_b, word_c
    word_a = words[word_a]
    word_b = words[word_b]
    word_c = words[word_c]
    e_a, e_b, e_c = word_to_vec[word_a], word_to_vec[word_b], word_to_vec[word_c]
    
    # max_cosine_sim = -999              
    # best_word = None                  


    # Compute cosine similarity between the vectors u and v
    #u:(e_b - e_a) 
    #v:((w's vector representation) - e_c)
    cosine_sim = find_cosine_similarity(word_to_vec, e_b - e_a + e_c)
    cosine_sim[word_a] = -float("Inf")
    cosine_sim[word_b] = -float("Inf")
    cosine_sim[word_c] = -float("Inf")

    max_index = torch.argmax(cosine_sim)
    max_index = int(max_index)
    for word in words:
        if words[word] == max_index:
            best_word = word
            break
        
    return best_word


# for taking input from the user and doing word analogy task on that
def take_input():
    # print('a --> b :: c --> d')
    # print('Enter a, b, c words separated by space')
    # words = input().split(' ')
    
    # best_pick = find_analogy(*words, word_to_vec)
    # print ('{} -> {} :: {} -> {}'.format( *words, best_pick))
    # print('Best pick: ' + best_pick)
    inp = []
    out = []
    with open("gdrive/MyDrive/NLP-2021/Project1-NLP/WordAnalogy-GLoVe/data/data.txt") as f:
      lines = f.readlines()
      lines = random.sample(lines, 100)
      for line in lines:
        if line[0].isalpha():
          line = line[:-1]
          data = line.split()
          for i in range(len(data)):
            data[i] = data[i].lower()
          inp.append(data[:3])
          out.append(data[3])
    f.close()
    correct = 0
    total = 0
    for i in range(len(inp)):
      best_pick = find_analogy(*inp[i], words, word_to_vec)
      if best_pick == out[i].lower():
        correct += 1
      total += 1
    print("Correct:", correct)
    print("Total:", total)
    f = open("gdrive/MyDrive/NLP-2021/Project1-NLP/WordAnalogy-GLoVe/output.txt", "w")
    f.write(str(correct/total))
    f.close()
take_input()

Correct: 73
Total: 100


In [ ]:
#Testing cells
import torch
my_tensor = torch.Tensor([[0.7185, 0.2953, 0.6841, 0.1045],
        [0.6817, 0.4053, 0.2318, 0.1309],
        [0.8265, 0.8029, 0.3165, 0.2020],
        [0.6041, 0.0118, 0.8386, 0.5076],
        [0.6985, 0.7313, 0.4613, 0.1862],
        [0.5678, 0.4485, 0.4514, 0.7747]])
tensor2 = torch.Tensor([1,2,3,4]).unsqueeze(0)
tensor3 = torch.Tensor([4])
tensor4 = torch.sum(tensor3.unsqueeze(1),dim=1)
print(tensor2/tensor3)
print(tensor3.unsqueeze(1)/torch.sum(tensor3.unsqueeze(1),dim=1).unsqueeze(1))
# print(tensor4.shape)
# print(tensor2.shape)
# print(torch.matmul(my_tensor, torch.transpose(tensor2, 0, 1)) /  torch.transpose(tensor3, 0, 1))
# print(torch.transpose(tensor2, 0, 1) / tensor2)

# print(my_tensor[0].shape)
# print(sum([0.7185, 0.2953, 0.6841, 0.1045]))

tensor([[0.2500, 0.5000, 0.7500, 1.0000]])
tensor([[1.]])


# Word Analogy (GLoVe) on other model
In this section, we use the 300-dimension vectors trained based on GLoVe model to assess the accuracy, and based on this, we compare our model's accuracy to this model.

In [ ]:

# coding: utf-8

# # <u>Word Analogy using Word Embeddings 
# In the word analogy task, we have $"a is to b as c is to __"$. For example is 'boy is to girl as king is to queen' .
# 
# We find a word $d$, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. <br>
# For finding $d$ we measure the similarity between $e_b - e_a$ and $e_d - e_c$ using **cosine** similarity. 
import os
import numpy as np
import random
os.chdir("gdrive/MyDrive/NLP-2021/Project1-NLP/WordAnalogy-GLoVe/")
from utility import *
os.chdir("../../../../../")

# For this task we will be using GLoVe Word Embeddings. Let us load that up.
# We will have
# - `words`: set of words in the vocabulary.
# - `word_to_vec`: dict mapping words to their GloVe vector representation.

words, word_to_vec = load_glove_vectors('gdrive/MyDrive/NLP-2021/Project1-NLP/WordAnalogy-GLoVe/data/glove.6B.300d.txt')


# ### Approach
# We will be using **cosine** similarity for finding the suitable word. We will use $e_b - e_a$ and $e_d - e_c$ as the two vectors to find their cosine, where $e_d$ is searched from all the other words in the vocabulary.
# 
# Given two vectors $u$ and $v$, cosine similarity is defined as follows: 
# 
# $$\text{Cosine Similarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) $$
# 
# where $u.v$ is the dot product of two vectors, $||u||_2$ is the norm of the vector $u$, and $\theta$ is the angle between $u$ and $v$.
# <br>Norm of $u$ is defined as $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$
# 
# This similarity depends on the angle between $u$ and $v$.
# <br>If $u$ and $v$ are very similar, their cosine similarity will be close to 1<br>
# If they are dissimilar, the cosine similarity will take a smaller value. 

# finds the cosine similarity between u and v
'''
    Arguments:
        u(n,) - vector of words            
        v(n,) - vector of words 
    Returns:
        cosine_sim - the cosine similarity between u and v
'''
def find_cosine_similarity(u, v):
    distance = 0.0
    
    # find the dot product between u and v 
    dot = np.dot(u,v)
    # find the L2 norm of u 
    norm_u = np.sqrt(np.sum(u**2))
    # Compute the L2 norm of v
    norm_v = np.sqrt(np.sum(v**2))
    # Compute the cosine similarity
    cosine_sim = dot/(norm_u)/norm_v
    
    return cosine_sim


# does the Word analogy task: a is to b as c is to ____
def find_analogy(word_a, word_b, word_c, word_to_vec):
    # convert words to lower case
    word_a = word_a.lower()
    word_b = word_b.lower()
    word_c = word_c.lower()
    
    
    # find the word embeddings for word_a, word_b, word_c
    e_a, e_b, e_c = word_to_vec[word_a], word_to_vec[word_b], word_to_vec[word_c]
    
    words = word_to_vec.keys()
    max_cosine_sim = -999              
    best_word = None                  

    # search for word_d in the whole word vector set
    for w in words:        
        # ignore input words
        if w in [word_a, word_b, word_c] :
            continue

        # Compute cosine similarity between the vectors u and v
        #u:(e_b - e_a) 
        #v:((w's vector representation) - e_c)
        cosine_sim = find_cosine_similarity(e_b - e_a, word_to_vec[w] - e_c)
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            # update word_d
            best_word = w
        
    return best_word



# for taking input from the user and doing word analogy task on that
def take_input():
    # print('a --> b :: c --> d')
    # print('Enter a, b, c words separated by space')
    # words = input().split(' ')
    
    # best_pick = find_analogy(*words, word_to_vec)
    # print ('{} -> {} :: {} -> {}'.format( *words, best_pick))
    # print('Best pick: ' + best_pick)
    inp = []
    out = []
    with open("gdrive/MyDrive/NLP-2021/Project1-NLP/WordAnalogy-GLoVe/data/data.txt") as f:
      lines = f.readlines()
      lines = random.sample(lines, 2)
      for line in lines:
        if line[0].isalpha():
          line = line[:-1]
          data = line.split()
          for i in range(len(data)):
            data[i] = data[i].lower()
          inp.append(data[:3])
          out.append(data[3])
    f.close()
    correct = 0
    total = 0
    for i in range(len(inp)):
      best_pick = find_analogy(*inp[i], word_to_vec)
      if best_pick == out[i]:
        correct += 1
      total += 1
    print("Correct:", correct)
    print("Total:", total)
    f = open("gdrive/MyDrive/NLP-2021/Project1-NLP/WordAnalogy-GLoVe/output.txt", "w")
    f.write(str(correct/total))
    f.close()

def main():
    take_input()

if __name__ == main():
    main()


Correct: 0
Total: 2


# Word similarity (SimVerb-3500)

In [ ]:
import os
print(os.getcwd())

/content/gdrive/My Drive/NLP-2021/Project1-NLP


In [ ]:
import os
import pandas as pd
import math
import numpy as np
from utility import *

words, word_to_vec = load_glove_vectors('WordAnalogy-GLoVe/data/glove.6B.300d.txt')


def find_cosine_similarity(u, v):
    distance = 0.0
    
    # find the dot product between u and v 
    dot = np.dot(u,v)
    # find the L2 norm of u 
    norm_u = np.sqrt(np.sum(u**2))
    # Compute the L2 norm of v
    norm_v = np.sqrt(np.sum(v**2))
    # Compute the cosine similarity
    cosine_sim = dot/(norm_u)/norm_v
    
    return cosine_sim

df = pd.read_csv("WordSimilarity/simverb-3500.csv")
word1 = list(df.loc[:,"word1"])
word2 = list(df.loc[:,"word2"])
result = list(df.loc[:, 'similarity'])
maxi = max(result)
for i in range(len(result)):
  result[i] /= maxi

error = 0
count = 0
for i in range(len(word1)):
  try:
    error += (find_cosine_similarity(word_to_vec[word1[i]], word_to_vec[word2[i]]) - result[i])**2
    count+=1
  except:
    pass
error = math.sqrt(error/count)
print(error)

0.3108337593595712


In [ ]:
import itertools
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm

# Drawing the embeddings
import matplotlib.pyplot as plt

# Deep learning:
from keras.models import Input, Model
from keras.layers import Dense

from scipy import sparse

# Custom functions
from utility import text_preprocessing, create_unique_word_dict

# Reading the text from the input folder
f = open("input/text8_short3", "r")
texts = f.readlines()
f.close()


# Defining the window for context
window = 5

# Creating a placeholder for the scanning of the word list
word_lists = []
all_text = []

for text in texts:

    # Cleaning the text
    text = text_preprocessing(text)

    # Appending to the all text list
    all_text += text

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text):
                word_lists.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])

unique_word_dict = create_unique_word_dict(all_text)

# Defining the number of features (unique words)
n_words = len(unique_word_dict)

# Getting all the unique words
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

# Converting the matrices into a sparse format because the vast majority of the data are 0s
X = sparse.csr_matrix(X)
Y = sparse.csr_matrix(Y)

# Defining the size of the embedding
embed_size = 300

# Defining the neural network
inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Optimizing the network weights
model.fit(
    x=X,
    y=Y,
    batch_size=64
    )

# Obtaining the weights from the neural network.
# These are the so called word embeddings

# The input layer
weights = model.get_weights()[0]

# Creating a dictionary to store the embeddings in. The key is a unique word and
# the value is the numeric vector
embedding_dict = {}
for word in words:
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

# Ploting the embeddings
# plt.figure(figsize=(10, 10))
# for word in list(unique_word_dict.keys()):
#     coord = embedding_dict.get(word)
#     plt.scatter(coord[0], coord[1])
#     plt.annotate(word, (coord[0], coord[1]))
# plt.show()
# Saving the embedding vector to a txt file
try:
    os.mkdir(f'{os.getcwd()}\\output')
except Exception as e:
    print(f'Cannot create output folder: {e}')

with open(f'{os.getcwd()}\\output\\embedding.txt', 'w') as f:
    for key, value in embedding_dict.items():
        try:
            f.write(f'{key}: {value}\n')
        except Exception as e:
            print(f'Cannot write word {key} to dict: {e}')

In [ ]:
# import itertools
# import pandas as pd
# import numpy as np
# import re
# import os
# from tqdm import tqdm

# # Drawing the embeddings
# import matplotlib.pyplot as plt

# # Deep learning:
# from keras.models import Input, Model
# from keras.layers import Dense

# from scipy import sparse

# # Custom functions
# from utility import text_preprocessing, create_unique_word_dict

# # Reading the text from the input folder
# f = open("input/text8_short3", "r")
# texts = f.readlines()
# f.close()


# # Defining the window for context
# window = 5

# # Creating a placeholder for the scanning of the word list
# word_lists = []
# all_text = []

# for text in texts:

#     # Cleaning the text
#     text = text_preprocessing(text)

#     # Appending to the all text list
#     all_text += text

#     # Creating a context dictionary
#     for i, word in enumerate(text):
#         for w in range(window):
#             # Getting the context that is ahead by *window* words
#             if i + 1 + w < len(text):
#                 word_lists.append([word] + [text[(i + 1 + w)]])
#             # Getting the context that is behind by *window* words
#             if i - w - 1 >= 0:
#                 word_lists.append([word] + [text[(i - w - 1)]])

# unique_word_dict = create_unique_word_dict(all_text)

# # Defining the number of features (unique words)
# n_words = len(unique_word_dict)

# # Getting all the unique words
# words = list(unique_word_dict.keys())

# # Creating the X and Y matrices using one hot encoding
# X = []
# Y = []

# for i, word_list in tqdm(enumerate(word_lists)):
#     # Getting the indices
#     main_word_index = unique_word_dict.get(word_list[0])
#     context_word_index = unique_word_dict.get(word_list[1])

#     # Creating the placeholders
#     X_row = np.zeros(n_words)
#     Y_row = np.zeros(n_words)

#     # One hot encoding the main word
#     X_row[main_word_index] = 1

#     # One hot encoding the Y matrix words
#     Y_row[context_word_index] = 1

#     # Appending to the main matrices
#     X.append(X_row)
#     Y.append(Y_row)

# # Converting the matrices into a sparse format because the vast majority of the data are 0s
# X = sparse.csr_matrix(X)
# Y = sparse.csr_matrix(Y)

# # Defining the size of the embedding
# embed_size = 300

# # Defining the neural network
# inp = Input(shape=(X.shape[1],))
# x = Dense(units=embed_size, activation='linear')(inp)
# x = Dense(units=Y.shape[1], activation='softmax')(x)
# model = Model(inputs=inp, outputs=x)
# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# # Optimizing the network weights
# model.fit(
#     x=X,
#     y=Y,
#     batch_size=64
#     )

# # Obtaining the weights from the neural network.
# # These are the so called word embeddings

# # The input layer
# weights = model.get_weights()[0]

# # Creating a dictionary to store the embeddings in. The key is a unique word and
# # the value is the numeric vector
# embedding_dict = {}
# for word in words:
#     embedding_dict.update({
#         word: weights[unique_word_dict.get(word)]
#         })

# # Ploting the embeddings
# # plt.figure(figsize=(10, 10))
# # for word in list(unique_word_dict.keys()):
# #     coord = embedding_dict.get(word)
# #     plt.scatter(coord[0], coord[1])
# #     plt.annotate(word, (coord[0], coord[1]))
# # plt.show()
# # Saving the embedding vector to a txt file
# try:
#     os.mkdir(f'{os.getcwd()}\\output')
# except Exception as e:
#     print(f'Cannot create output folder: {e}')

# with open(f'{os.getcwd()}\\output\\embedding.txt', 'w') as f:
#     for key, value in embedding_dict.items():
#         try:
#             f.write(f'{key}: {value}\n')
#         except Exception as e:
#             print(f'Cannot write word {key} to dict: {e}')



65920it [00:01, 64513.31it/s]


TypeError: ignored